<a href="https://colab.research.google.com/github/Palak2506/ML-project/blob/main/Facebook_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Importing Dataset

In [9]:
df = pd.read_csv('Facebook_Marketplace_data.csv')
X = df.iloc[:,:-1].values #All columns except the last
y = df.iloc[:,-1].values   #Only last column

In [10]:
print(X)

[[1 'video' '4/22/2018 6:00' ... nan nan nan]
 [2 'photo' '4/21/2018 22:45' ... nan nan nan]
 [3 'video' '4/21/2018 6:17' ... nan nan nan]
 ...
 [7048 'photo' '9/21/2016 23:03' ... nan nan nan]
 [7049 'photo' '9/20/2016 0:43' ... nan nan nan]
 [7050 'photo' '9/10/2016 10:30' ... nan nan nan]]


In [11]:
print(y)

[nan nan nan ... nan nan nan]


Encoding Categorical values

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.